<a href="https://colab.research.google.com/github/Sabastain-Wakoyi/CS-6263-HW5/blob/main/CS_ASS5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch pandas

In [2]:
!pip install scikit-learn

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import load_dataset

# Load dataset
dataset = load_dataset("financial_phrasebank", "sentences_50agree")

# Convert to pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)  # 10% for testing

# Converting back to datasets format if needed
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [9]:
def add_instructions(example):
    instruction = "Determine if the sentiment of the following financial statement is positive, negative, or neutral: "
    example['sentence'] = instruction + example['sentence']
    return example

train_dataset = train_dataset.map(add_instructions)
test_dataset = test_dataset.map(add_instructions)


Map:   0%|          | 0/4361 [00:00<?, ? examples/s]

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

In [7]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
dataset = load_dataset("financial_phrasebank", "sentences_50agree")
print(dataset)

# Add instructions to each data instance
def add_instructions(row):
    instruction = "Determine if the sentiment of the following financial statement is positive, negative, or neutral: "
    row['sentence'] = instruction + row['sentence']
    return row

# Apply the modification
modified_dataset = dataset.map(add_instructions)


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4846
    })
})


In [4]:
!pip install transformers[torch] -U

In [12]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize_data(example):
    return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=128)

train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)


Map:   0%|          | 0/4361 [00:00<?, ? examples/s]

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

In [19]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def add_instructions(example):
    instruction = "Determine if the sentiment of the following financial statement is positive, negative, or neutral: "
    example['sentence'] = instruction + example['sentence']
    return example

def tokenize_data(example):
    return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=128)

# Add instructions and tokenize the data
train_dataset = train_dataset.map(add_instructions).map(tokenize_data, batched=True)
test_dataset = test_dataset.map(add_instructions).map(tokenize_data, batched=True)


Map:   0%|          | 0/4361 [00:00<?, ? examples/s]

Map:   0%|          | 0/4361 [00:00<?, ? examples/s]

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

In [14]:
# Fine-tune the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.223300,0.736149
2,0.111900,0.758855
3,0.094500,0.784218


TrainOutput(global_step=1638, training_loss=0.12455441562233718, metrics={'train_runtime': 69.2939, 'train_samples_per_second': 188.805, 'train_steps_per_second': 23.638, 'total_flos': 433275470869248.0, 'train_loss': 0.12455441562233718, 'epoch': 3.0})

In [16]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Initialize the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch"
)

# Initialize the Trainer with correct datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.677200,0.438894
2,0.388200,0.456205
3,0.218500,0.594671


TrainOutput(global_step=1638, training_loss=0.40670736077649833, metrics={'train_runtime': 69.003, 'train_samples_per_second': 189.6, 'train_steps_per_second': 23.738, 'total_flos': 433275470869248.0, 'train_loss': 0.40670736077649833, 'epoch': 3.0})

In [18]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.5946707725524902, 'eval_runtime': 0.7037, 'eval_samples_per_second': 689.23, 'eval_steps_per_second': 86.687, 'epoch': 3.0}


In [17]:
model.save_pretrained('./fine_tuned_distilbert')
